In [2]:
import numpy as np 
from numpy import linalg as LA
import pandas as pd 
from matplotlib import pyplot as plt 
from numba import jit
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error



""" Angular Momentum Operators """
@jit
def Jz(bra,ket):
    m = float(ket)
    if bra == ket:
        return m
    else: 
        return 0
@jit
def JJz(J_power,Jz_power,ket):     # For Jz^a[J(J+1)]^b
    j=7.5
    m=float(ket)
    return(((j*(j+1))**J_power)*(m**Jz_power))

@jit
def Jp(power,bra,ket):
    j=7.5
    ket=float(ket)
    bra=float(bra)
    m=ket
    jp=1
    
    if (ket+power == bra):
        for i in range(power):
            jp*= np.sqrt( j*(j+1) - (m+i)*(m+1+i) )
            ket+=1
    else: 
        jp=0
    
    return(jp,ket) 
@jit   
def Jm(power,bra,ket):
    j=7.5
    ket=float(ket)
    bra=float(bra)
    m=ket
    
    jm=1
    
    if (ket-power == bra):
        for i in range(power):
            jm*= np.sqrt( j*(j+1) - (m-i)*(m-1-i) )
            ket-=1
    else: 
        jm=0
    
    return(jm,ket) 

""" Steven's Operator functions """
@jit
def S_02(bra,ket): 
    s=3*JJz(0,2,ket)-JJz(1,0,ket)
    
    return s*(int(bra==ket))

@jit
def S_04(bra,ket):
    s=35*JJz(0,4,ket)-30*JJz(1,2,ket)+25*JJz(0,2,ket)-6*JJz(1,0,ket)+3*JJz(2,0,ket)
    return(s*(int(bra==ket)))

@jit
def S_06(bra,ket):
    s=231*JJz(0,6,ket)-315*JJz(1,4,ket)+735*JJz(0,4,ket)+105*JJz(2,2,ket)-525*JJz(1,2,ket)+294*JJz(0,2,ket)-5*JJz(3,0,ket)+40*JJz(2,0,ket)-60*JJz(1,0,ket)
    return(s*(int(bra==ket)))

@jit
def S_66(bra,ket):
    return( 0.5*(Jp(6,bra,ket)[0]+Jm(6,bra,ket)[0])) 

@jit
def S_34(bra,ket):
    
    a=JJz(0,1,Jp(3,bra,ket)[1])*Jp(3,bra,ket)[0] + JJz(0,1,Jm(3,bra,ket)[1])*Jm(3,bra,ket)[0]   
    a+= JJz(0,1,ket)*(Jp(3,bra,JJz(0,1,ket))[0] + Jm(3,bra,JJz(0,1,ket))[0])
    
    return 0.25*a

@jit
def S_36(bra,ket): 
    
    a=(11*JJz(0,3,ket)-3*JJz(1,1,ket)-59*JJz(0,1,ket))*(Jp(3,bra,ket)[0]+Jm(3,bra,ket)[0])
    
    ket_p=Jp(3,bra,ket)[1]
    ket_m=Jm(3,bra,ket)[1]
    
    a+=Jp(3,bra,ket)[0]*((11*JJz(0,3,ket_p)-3*JJz(1,1,ket_p)-59*JJz(0,1,ket_p))) 
    a+=Jm(3,bra,ket)[0]*((11*JJz(0,3,ket_m)-3*JJz(1,1,ket_m)-59*JJz(0,1,ket_m)))
    
    return a*0.25


""" Hamiltonian Calculation from Steven's operator parameters """

#@jit

def Matrix(B): 
    M=np.zeros([16,16], dtype=float)    
    print(type(M))
    #B=[1.135,-0.0615,0.0011,0.005,0.315,0.037]
    
    for i in range(16):
        for j in range(16):
            ket=i-7.5
            bra=j-7.5
            if i==j:
                M[i][j]= B[0]*S_02(bra,ket)+B[1]*S_04(bra,ket)+B[2]*S_06(bra,ket)
            else:
                M[i][j]= B[3]*S_66(bra,ket)+B[4]*S_34(bra,ket)+B[5]*S_36(bra,ket)  
    return(M)
            


""" Eigen System Calculation  """ 
@jit
def eigenfunc(a):
    return(np.linalg.eigh(a)) 


""" Anisotropy Calculation """

def jz(gs_1):
    result = 0
    for i in range(16):
        m = 7.5-i
        result += m*gs_1[i]**2
    return result

def jpp(gs_1, gs_2):
    J = 7.5
    temp = np.zeros(16)
    for i in range(15):
        m = 7.5 - i
        temp[i+1] = np.sqrt(J*(J+1)-m*(m+1))*gs_1[i]
    gs_1 = temp
    return(np.dot(gs_1,gs_2))

def anisotropy(gs_1,gs_2):
    return 2*jz(gs_1)/jpp(gs_1,gs_2)

""" Constraint """


def constraint(parameter):
    gs_1 = np.round(eigenfunc(Matrix(parameter))[1][:,0],3)
    gs_2 = np.round(eigenfunc(Matrix(parameter))[1][:,1],3)
    
    g = g_avg(gs_1,gs_2)
    if g>0.5: 
        return 100000*(g-0.5)
    else: 
        return 0.0


def g_avg(gs_1, gs_2):
    return np.sqrt( 0.33*(jpp(gs_1,gs_2)**2 + 2*jz(gs_1)**2 ) )

""" Error metric For Er """

def err_ER(parameters):
    from_experiment = B
    calculated = eigenfunc(Matrix(parameters))[0]
    calculated = np.sort(calculated)
    return np.sqrt(mean_squared_error(from_experiment, calculated)) 
@jit
def err_ER_2(parameters):
    from_experiment = B
    eigensystem = eigenfunc(Matrix(parameters))
    calculated = eigensystem[0]
    calculated = np.sort(calculated)
    
    gs_1 = eigensystem[1][0]
    gs_2 = eigensystem[1][1]
    
    calc_g = 100*np.sqrt( 0.33*(jpp(gs_1,gs_2)**2 + 2*jz(gs_1)**2) )
    expt_g = 100*0.4
    
    return np.sqrt(mean_squared_error(from_experiment, calculated) 
                   + mean_squared_error(parameters, theo_param)) # + constraint(parameters)





T = 0

B = [0, 0, 1.25, 1.25, 1.55, 1.55, 4.9, 4.9, 8.27, 8.27 ,9.94, 9.94, 11.37, 11.37, 13.17, 13.17]
#B = [0, 0, 1.25, 1.25, 1.55, 1.55, 4.9, 4.9, 9.94, 9.94,11.37, 11.37 , 13.17, 13.17, 200, 200]

theo_param = [1.270,-0.0372,0.00025,0.0024,0.275,0.0023]


In [3]:
# Fit Function Definition 
""" Minimisation using randomised algorithm : Basin Hopping, Use with  Powell minimizer """

from scipy.optimize import basinhopping 
from scipy.optimize import minimize 

def fit(paper = [1.270,-0.0372,0.00025,0.0024,0.275,0.0023], number_of_iterations=5):
#b =[1.135,-0.0615,0.0011,0.005,0.315,0.037] +  0.5*np.ones(6)
    
    b = paper
    return basinhopping(err_ER_2, b, niter = number_of_iterations, minimizer_kwargs ={ 'method' : 'Powell'})

def fit(paper = [1.270,-0.0372,0.00025,0.0024,0.275,0.0023],number_of_iterations= 30,test = 'True'):
#b =[1.135,-0.0615,0.0011,0.005,0.315,0.037] +  0.5*np.ones(6)
    b = paper
    #return basinhopping(err_ER_2, b, niter = number_of_iterations, minimizer_kwargs ={ 'method' : 'Powell'})
    return minimize(err_ER_2, b, args=(), method='BFGS', 
                    tol=None, callback=None, 
                    options={ 'xtol': 0.0000001, 'ftol': 0.0000001, 
                             'maxiter': 100000, 'maxfev': None, 'disp': False, 'direc': None, 'return_all': False})

    
 

In [4]:
# Fit call; Takes ~2mins for 30 iterations 
from IPython.display import clear_output
result = fit(test = 'True')
clear_output()

In [5]:


print(":: Result Error :: ")
print(err_ER(result.x))

print("result.x = ", result.x)
# Printing details 

res = eigenfunc(Matrix(result.x))
t= res[0] 
t =t - min(t)
print("Calculated Eigen Energies")
print(np.round(t,4))
print("Experimental Eigen Energies")
print(B)

print("Anisotropy")
gs_1 = np.round(eigenfunc(Matrix(result.x))[1][:,0],3)
gs_2 = np.round(eigenfunc(Matrix(result.x))[1][:,1],3)



print(np.sqrt(( 0.33*( jpp(gs_2,gs_1)**2 + 2*jz(gs_1)**2 ))))

#print(np.abs(anisotropy(gs_1,gs_2)))

print(12/30)

:: Result Error :: 
<class 'numpy.ndarray'>
6.309044726676186
result.x =  [ 2.13591193e-02 -4.45422650e-04 -4.98401541e-08 -2.29868010e-08
  5.28545461e-05 -4.13440726e-07]
<class 'numpy.ndarray'>
Calculated Eigen Energies
[ 0.      0.      1.2471  1.2471  1.6974  1.6974  4.7385  4.7385  8.4081
  8.4081 10.2039 10.2039 11.6126 11.6126 12.8555 12.8555]
Experimental Eigen Energies
[0, 0, 1.25, 1.25, 1.55, 1.55, 4.9, 4.9, 8.27, 8.27, 9.94, 9.94, 11.37, 11.37, 13.17, 13.17]
Anisotropy
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
0.4059872923531413
0.4


## Data From SQUID

\begin{align}
M_{saturation}  = n\mu_{B}gJ \\
\end{align}

\begin{align}
M_{saturation} = 6.1 (approx),
J = \frac{15}{2} \\ 
\end{align}

This gives us $g=0.4$ 

In [6]:
print("Experimental Eigen Energies ::\n",B)
print("Calculated Eigen Energies ::\n",np.round(t,3))

print("g_mean from SQUID ::", 0.4, ", g-mean calculated ::",np.round(np.sqrt(( 0.33*( jpp(gs_2,gs_1)**2 + 2*jz(gs_1)**2))),3))

print("Anisotropy ::", np.round(anisotropy(gs_1,gs_2),3))

print(" :: Ground_State (A very dominating 1/2 contribution) :: \n")

for i in range(16):
    print((i-15/2), end = "|")
print('\n')
print(*np.round(gs_2,2),)

Experimental Eigen Energies ::
 [0, 0, 1.25, 1.25, 1.55, 1.55, 4.9, 4.9, 8.27, 8.27, 9.94, 9.94, 11.37, 11.37, 13.17, 13.17]
Calculated Eigen Energies ::
 [ 0.     0.     1.247  1.247  1.697  1.697  4.738  4.738  8.408  8.408
 10.204 10.204 11.613 11.613 12.855 12.855]
g_mean from SQUID :: 0.4 , g-mean calculated :: 0.406
Anisotropy :: 0.126
 :: Ground_State (A very dominating 1/2 contribution) :: 

-7.5|-6.5|-5.5|-4.5|-3.5|-2.5|-1.5|-0.5|0.5|1.5|2.5|3.5|4.5|5.5|6.5|7.5|

-0.0 0.0 0.0 -0.0 0.0 0.01 -0.0 0.02 1.0 -0.0 -0.0 -0.01 0.0 0.0 0.0 0.0


In [30]:
def Sqw(eigen_system):   
    
    eig_val = eigen_system[0]
    eig_vect = eigen_system[1]
    
    intensities = np.zeros(7)
    
    for i in range(7):
        intensities[i] = Intensities(eig_vect[:,i*2+3], eig_vect[:,i*2+2], eig_vect[:,1], eig_vect[:,0])
    return intensities
    
    
def Intensities(a_1,b_1,a_0,b_0):
    sum =0
    jx=0
    ijy=0
    jz=0
    for i in np.arange(16):
        for j in np.arange(16):
            jx += (Jm(1,7.5-i,7.5-j)[0]+Jp(1,7.5-i,7.5-j)[0])*a_1[j]*a_0[i]/2
            ijy += (-Jm(1,7.5-i,7.5-j)[0]+Jp(1,7.5-i,7.5-j)[0])*a_1[j]*a_0[i]/2
            jz += Jz(7.5-i,7.5-j)*a_1[j]*a_0[i]
            
    sum = jx**2 + (ijy)**2 + jz**2
    
    jx=0
    ijy=0
    jz=0
    for i in np.arange(16):
        for j in np.arange(16):
            jx += (Jm(1,7.5-i,7.5-j)[0]+Jp(1,7.5-i,7.5-j)[0])*b_1[j]*a_0[i]/2
            ijy += (-Jm(1,7.5-i,7.5-j)[0]+Jp(1,7.5-i,7.5-j)[0])*b_1[j]*a_0[i]/2
            jz += Jz(7.5-i,7.5-j)*b_1[j]*a_0[i]
            
    sum += jx**2 + (ijy)**2 + jz**2
    
    jx=0
    ijy=0
    jz=0
    for i in np.arange(16):
        for j in np.arange(16):
            jx += (Jm(1,7.5-i,7.5-j)[0]+Jp(1,7.5-i,7.5-j)[0])*a_1[j]*b_0[i]/2
            ijy += (-Jm(1,7.5-i,7.5-j)[0]+Jp(1,7.5-i,7.5-j)[0])*a_1[j]*b_0[i]/2
            jz += Jz(7.5-i,7.5-j)*a_1[j]*b_0[i]
            
    sum += jx**2 + (ijy)**2 + jz**2 
    jx=0
    ijy=0
    jz=0
    for i in np.arange(16):
        for j in np.arange(16):
            jx += (Jm(1,7.5-i,7.5-j)[0]+Jp(1,7.5-i,7.5-j)[0])*b_1[j]*b_0[i]/2
            ijy += (-Jm(1,7.5-i,7.5-j)[0]+Jp(1,7.5-i,7.5-j)[0])*b_1[j]*b_0[i]/2
            jz += Jz(7.5-i,7.5-j)*b_1[j]*b_0[i]
            
    sum += jx**2 + (ijy)**2 + jz**2
            
            
            
    return (sum)
    
   

In [31]:
trial = eigenfunc(Matrix(result.x))
Sqw(trial)

<class 'numpy.ndarray'>


array([8.98112486e-05, 6.29992805e+01, 7.61386068e-04, 6.09687837e-04,
       8.96748915e-07, 1.24237774e-06, 1.85144383e-06])